In [1]:
project_id = 'elife-data-pipeline'
source_dataset = 'de_dev'
output_dataset = 'de_dev'
output_table_prefix = 'data_science_'
manuscript_min_tf = 10
manuscript_max_tf = 0.9
state_path = 's3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev'

In [2]:
import os
from functools import partial
from itertools import groupby
from typing import List, Tuple, TypeVar

import joblib
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import data_science_pipeline.configure_warnings

from data_science_pipeline.sql import get_sql
from data_science_pipeline.utils.bq import to_gbq
from data_science_pipeline.utils.io import load_object_from
from data_science_pipeline.utils.misc import identity_fn
from data_science_pipeline.utils.jupyter import (
    read_big_query as _read_big_query,
)

In [3]:
model_path = os.path.join(state_path, 'senior_editor_model.joblib')
recommendation_output_table_name = '{output_dataset}.{prefix}{suffix}'.format(
    output_dataset=output_dataset,
    prefix=output_table_prefix,
    suffix='editor_recommendation'
)

In [4]:
print('loading model from:', model_path)
model_dict = load_object_from(model_path)
model_dict.keys()

loading model from: s3://ci-elife-data-pipeline/airflow-config/keyword-extraction/state-dev/senior_editor_model.joblib


dict_keys(['editor_tf_idf_vectorizer', 'doc_length_weighted_editor_tf_idf', 'editor_names', 'editor_person_ids'])

In [5]:
editor_tf_idf_vectorizer = model_dict['editor_tf_idf_vectorizer']
editor_tf_idf_vectorizer

TfidfVectorizer(lowercase=False, smooth_idf=False, token_pattern=None,
                tokenizer=<function identity_fn at 0x7f4a8665de60>)

In [6]:
doc_length_weighted_editor_tf_idf = model_dict['doc_length_weighted_editor_tf_idf']
doc_length_weighted_editor_tf_idf

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00139157, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00150745, 0.00109185, 0.        , ..., 0.        , 0.00292359,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.01753493,
         0.        ],
        [0.        , 0.0010018 , 0.        , ..., 0.00068432, 0.00076642,
         0.        ],
        [0.00065092, 0.00141439, 0.        , ..., 0.        , 0.00216414,
         0.        ]])

In [7]:
editor_names = model_dict['editor_names']
editor_names

0           Aleksandra Walczak
1                  Andrew King
2               Anna Akhmanova
3     Barbara Shinn-Cunningham
4                Carla Rothlin
                ...           
60                 Tamar Makin
61             Timothy Behrens
62              Utpal Banerjee
63              Vivek Malhotra
64               Wendy Garrett
Name: name, Length: 65, dtype: object

In [8]:
editor_person_ids = model_dict['editor_person_ids']
editor_person_ids

0     50904
1     14601
2      8518
3     19576
4     44396
      ...  
60    18331
61     1044
62     1042
63     1133
64    28627
Name: person_id, Length: 65, dtype: object

In [9]:
editor_person_id_by_name_map = dict(zip(editor_names, editor_person_ids))
editor_person_id_by_name_map

{'Aleksandra Walczak': '50904',
 'Andrew King': '14601',
 'Anna Akhmanova': '8518',
 'Barbara Shinn-Cunningham': '19576',
 'Carla Rothlin': '44396',
 'Catherine Dulac': '1014',
 'Chris Baker': '28129',
 'Christian Büchel': '16197',
 'Christian Hardtke': '1102',
 'Christian Rutz': '15332',
 'Clifford Rosen': '48282',
 'Cynthia Wolberger': '5005',
 'David Ron': '1174',
 'Detlef Weigel': '1030',
 'Didier Stainier': '7189',
 'Diethard Tautz': '1191',
 'Dominique Soldati-Favre': '41647',
 'Eduardo Franco': '1086',
 'Edward Morrisey': '55645',
 'Floris de Lange': '28130',
 'Gary Westbrook': '1202',
 'George Perry': '42011',
 'Gisela Storz': '1188',
 'Huda Zoghbi': '1029',
 'James Manley': '1020',
 'Jessica Tyler': '1421',
 'John Huguenard': '13947',
 'John Kuriyan': '1018',
 'Jonathan Cooper': '1062',
 'Jos van der Meer': '41027',
 'Joshua Gold': '17965',
 'José Faraldo-Gómez': '13987',
 'K VijayRaghavan': '1027',
 'Karla Kirkegaard': '3645',
 'Kate Wassum': '33127',
 'Kathryn Cheah': '91149

In [10]:
read_big_query = partial(_read_big_query, project_id=project_id)

In [11]:
default_query_props = dict(project=project_id, dataset=source_dataset)

In [12]:
manuscript_version_for_recommendation_df = read_big_query(
    get_sql('manuscript-version-initial-submissions-for-senior-editor-recommendation.sql').format(
        **default_query_props
    )
)
manuscript_version_for_recommendation_df.head()

> ```sql
> WITH t_manuscript_version_abstract_keywords AS (
>   SELECT
>     LPAD(CAST(manuscript_abstract_keywords.manuscript_id AS STRING), 5, '0') AS manuscript_id,
>     manuscript_abstract_keywords.version_id,
>     manuscript_abstract_keywords.extracted_keywords,
>     ROW_NUMBER() OVER (
>       PARTITION BY version_id
>       ORDER BY data_hub_imported_timestamp DESC
>     ) AS version_id_row_number
>   FROM `elife-data-pipeline.de_dev.manuscript_abstract_keywords` AS manuscript_abstract_keywords
>   WHERE ARRAY_LENGTH(extracted_keywords) > 0
> ),
> 
> t_last_manuscript_version_abstract_keywords AS (
>   SELECT
>     * EXCEPT(version_id_row_number)
>   FROM t_manuscript_version_abstract_keywords
>   WHERE version_id_row_number = 1
>   ORDER BY version_id
> )
> 
> SELECT version.version_id, manuscript_version_abstract_keywords.extracted_keywords
> FROM `elife-data-pipeline.de_dev.mv_manuscript_version` AS version
> JOIN t_last_manuscript_version_abstract_keywords AS manuscript_version_abstract_keywords
>   ON manuscript_version_abstract_keywords.version_id = version.version_id
> WHERE version.overall_stage = 'Initial Submission'
>   AND ARRAY_LENGTH(version.senior_editors) = 0
>   AND TIMESTAMP_DIFF(CURRENT_TIMESTAMP, version.created_timestamp, DAY) < 365
> ```

Downloading: 100%|██████████| 135/135 [00:01<00:00, 120.01rows/s]


,version_id,extracted_keywords
0,49586/2019-06-22T03:44:31Z,"[ac, advantage, affective, affective value, am..."
1,50199/2019-07-14T19:30:01Z,"[agent, cardiopulmonary, cardiopulmonary syndr..."
2,51204/2019-08-19T14:01:01Z,"[etiology, extent, impairment, infertility, le..."
3,51769/2019-09-10T09:20:03Z,"[3'-untranslate, 3'-untranslated region, 3'-ut..."
4,52332/2019-09-30T11:45:20Z,"[584e, 584e mutation, 592a, 592a mutation, aci..."


In [13]:
keyword_similarity = cosine_similarity(
    editor_tf_idf_vectorizer.transform(
        manuscript_version_for_recommendation_df
        ['extracted_keywords']
    ),
    doc_length_weighted_editor_tf_idf
)
print(keyword_similarity.max())
keyword_similarity

0.3355734086321363


array([[0.08724932, 0.27371087, 0.08635848, ..., 0.07420609, 0.07733345,
        0.07455245],
       [0.09197159, 0.04039304, 0.09755077, ..., 0.07793741, 0.10870541,
        0.12054067],
       [0.01058839, 0.01726941, 0.01816204, ..., 0.03753092, 0.01436975,
        0.01734019],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.10137202, 0.08209501, 0.09830257, ..., 0.07170343, 0.08062227,
        0.10260865],
       [0.11923865, 0.06587583, 0.11196174, ..., 0.08769475, 0.11397696,
        0.12454785]])

In [14]:
doc_length_weighted_editor_tf_idf.shape

(65, 20373)

In [15]:
doc_length_weighted_editor_tf_idf

matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00139157, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.00150745, 0.00109185, 0.        , ..., 0.        , 0.00292359,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.01753493,
         0.        ],
        [0.        , 0.0010018 , 0.        , ..., 0.00068432, 0.00076642,
         0.        ],
        [0.00065092, 0.00141439, 0.        , ..., 0.        , 0.00216414,
         0.        ]])

In [16]:
editor_tf_idf_vectorizer.transform(
    manuscript_version_for_recommendation_df
    ['extracted_keywords']
)[0]

<1x20373 sparse matrix of type '<class 'numpy.float64'>'
	with 71 stored elements in Compressed Sparse Row format>

In [17]:
def get_single_manuscript_matching_keywords(
        manuscript_keywords,
        editor_tf_idf=None):
    # Note: multiply is much faster with a dense matrix 
    matching_keyword_matrix = np.multiply(
        editor_tf_idf_vectorizer.transform(
            [manuscript_keywords]
        )[0].todense(),
        editor_tf_idf
    )
    non_zero_matrix = np.nonzero(matching_keyword_matrix)
    values = np.asarray(matching_keyword_matrix[non_zero_matrix])[0]
    flat_matching_keywords = np.array(editor_tf_idf_vectorizer.get_feature_names())[non_zero_matrix[-1]]
    d = {
        key: sorted((t[1] for t in grouped_values), reverse=True)
        for key, grouped_values in groupby(
            zip(non_zero_matrix[0], zip(values, flat_matching_keywords)),
            key=lambda t: t[0]
        )
    }
    return [d.get(i, []) for i in range(len(editor_tf_idf))]


get_single_manuscript_matching_keywords(
    manuscript_version_for_recommendation_df
    ['extracted_keywords']
    [0],
    editor_tf_idf=doc_length_weighted_editor_tf_idf[:3]
)

[[(0.033179822367744666, 'result'),
  (0.02779931063243472, 'different'),
  (0.026818455748117673, 'interaction'),
  (0.0197285430294698, 'approach'),
  (0.018686547290504042, 'behavior'),
  (0.017038287161814828, 'function'),
  (0.016645938050555798, 'control'),
  (0.016141535205929838, 'effect'),
  (0.015721163714413806, 'type'),
  (0.014348031294159855, 'individual'),
  (0.013351511485281926, 'specific'),
  (0.011477717942461047, 'fundamental'),
  (0.009536793918058518, 'signal'),
  (0.008114247619818177, 'information'),
  (0.006884517422817279, 'impact'),
  (0.006675755742640963, 'direct'),
  (0.006473420352993922, 'hypothesis'),
  (0.006276325242418765, 'survival'),
  (0.006085685714863632, 'question'),
  (0.005380511735309946, 'mouse'),
  (0.00459108717698442, 'advantage'),
  (0.004384106301186599, 'social'),
  (0.004287093653755298, 'cortical'),
  (0.004184216828279176, 'imaging'),
  (0.004153645894913619, 'input'),
  (0.003699097344567955, 'functional'),
  (0.003581802954667697

In [18]:
def get_multiple_manuscript_matching_keywords(
        manuscript_keywords_list: pd.Series,
        editor_tf_idf=None):
    return (
        manuscript_keywords_list
        .apply(partial(
            get_single_manuscript_matching_keywords,
            editor_tf_idf=editor_tf_idf
        ))
    )


get_multiple_manuscript_matching_keywords(
    manuscript_version_for_recommendation_df['extracted_keywords'][:3],
    editor_tf_idf=doc_length_weighted_editor_tf_idf
)

0    [[(0.033179822367744666, result), (0.027799310...
1    [[(0.07277482662334517, cell), (0.028519864487...
2    [[(0.012363999429937204, signaling), (0.011371...
Name: extracted_keywords, dtype: object

In [19]:
manuscript_matching_keywords_ser = get_multiple_manuscript_matching_keywords(
    manuscript_version_for_recommendation_df['extracted_keywords'],
    editor_tf_idf=doc_length_weighted_editor_tf_idf
)
manuscript_matching_keywords_ser.head()

0    [[(0.033179822367744666, result), (0.027799310...
1    [[(0.07277482662334517, cell), (0.028519864487...
2    [[(0.012363999429937204, signaling), (0.011371...
3    [[(0.038153578011224654, mechanism), (0.034576...
4    [[(0.032713358480555545, interaction), (0.0279...
Name: extracted_keywords, dtype: object

In [20]:
T = TypeVar('T')


def get_recommended_editors_with_probability(
        proba_matrix: List[List[float]],
        manuscript_matching_keywords_ser: pd.Series,
        indices: List[T],
        threshold: float = 0.5) -> List[List[Tuple[float, T]]]:
    return [
        sorted([
            (
                p,
                key,
                sum(
                    s for s, _ in editor_matching_keywords
                ),
                editor_matching_keywords
            )
            for p, key, editor_matching_keywords in zip(
                row,
                indices,
                editors_matching_keywords
            ) if p >= threshold
        ], reverse=True)
        for row, editors_matching_keywords in zip(proba_matrix, manuscript_matching_keywords_ser)
    ]


prediction_results_with_similarity = pd.Series(
    get_recommended_editors_with_probability(
        keyword_similarity,
        manuscript_matching_keywords_ser,
        editor_names,
        threshold=0.001
    ),
    index=manuscript_version_for_recommendation_df.index
)
# print(prediction_results_with_similarity[0])
prediction_results_with_similarity[:5]

0    [(0.2737108728328216, Andrew King, 0.235260319...
1    [(0.24877103013261137, Karla Kirkegaard, 0.888...
2    [(0.04398606884115364, Michael Eisen, 0.203095...
3    [(0.21636200978749964, James Manley, 0.1419143...
4    [(0.2509168632420775, Dominique Soldati-Favre,...
dtype: object

In [21]:
prediction_results_df = pd.concat([
    manuscript_version_for_recommendation_df['version_id'],
    prediction_results_with_similarity.to_frame('prediction'),
], axis=1)
print(len(prediction_results_df))
prediction_results_df.head()

135


,version_id,prediction
0,49586/2019-06-22T03:44:31Z,"[(0.2737108728328216, Andrew King, 0.235260319..."
1,50199/2019-07-14T19:30:01Z,"[(0.24877103013261137, Karla Kirkegaard, 0.888..."
2,51204/2019-08-19T14:01:01Z,"[(0.04398606884115364, Michael Eisen, 0.203095..."
3,51769/2019-09-10T09:20:03Z,"[(0.21636200978749964, James Manley, 0.1419143..."
4,52332/2019-09-30T11:45:20Z,"[(0.2509168632420775, Dominique Soldati-Favre,..."


In [22]:
prediction_results_df['prediction'][0]

[(0.2737108728328216,
  'Andrew King',
  0.23526031925375473,
  [(0.031254860363776144, 'auditory'),
   (0.02332372542637609, 'auditory cortex'),
   (0.01497986763189649, 'cortex'),
   (0.01092065821966457, 'neuron'),
   (0.008466535528416396, 'result'),
   (0.00814491733080403, 'cortical'),
   (0.007228748738562737, 'input'),
   (0.006619469685126562, 'information'),
   (0.006515720890532435, 'mouse'),
   (0.005185091044376566, 'mouse auditory cortex'),
   (0.005163878740751091, 'excitatory'),
   (0.0048414914945729024, 'principal neuron'),
   (0.00481357085069876, 'circuit'),
   (0.004481256698884687, 'signal'),
   (0.004018678154040963, 'function'),
   (0.004017879784736142, 'auditory information'),
   (0.003979532727068706, 'behavior'),
   (0.0038626129830102453, 'different'),
   (0.003784580397494887, 'effect'),
   (0.003741901802068501, 'functional'),
   (0.0036622098077616233, 'spike'),
   (0.0035937340912825942, 'projection'),
   (0.003502290060542743, 'inhibitory'),
   (0.0027

In [23]:
prediction_results_flat_df = pd.DataFrame([
    {
        'version_id': row.version_id,
        'score': predicted_editor[0],
        'name': predicted_editor[1],
        'person_id': editor_person_id_by_name_map[predicted_editor[1]],
        'matching_keyword_score': predicted_editor[2],
        'matching_keywords': [{
            'score': keyword_score,
            'keyword': keyword
        } for keyword_score, keyword in predicted_editor[3]],
    }
    for row in prediction_results_df.itertuples()
    for predicted_editor in row.prediction
])
print(len(prediction_results_flat_df))
prediction_results_flat_df.head()

8639


,version_id,score,name,person_id,matching_keyword_score,matching_keywords
0,49586/2019-06-22T03:44:31Z,0.273711,Andrew King,14601,0.235260,"[{'score': 0.031254860363776144, 'keyword': 'a..."
1,49586/2019-06-22T03:44:31Z,0.207845,Kate Wassum,33127,0.572228,"[{'score': 0.040050071942118654, 'keyword': 'n..."
2,49586/2019-06-22T03:44:31Z,0.205554,John Huguenard,13947,1.044847,"[{'score': 0.10105947912864989, 'keyword': 'ex..."
3,49586/2019-06-22T03:44:31Z,0.204885,Catherine Dulac,1014,0.090280,"[{'score': 0.008137041176363848, 'keyword': 'n..."
4,49586/2019-06-22T03:44:31Z,0.202564,Ronald Calabrese,1056,0.185415,"[{'score': 0.01876724566787909, 'keyword': 'ne..."


In [24]:
prediction_results_flat_df['version_id'].nunique()

134

In [25]:
print('writing to:', recommendation_output_table_name)
to_gbq(
    prediction_results_flat_df,
    recommendation_output_table_name,
    project_id=project_id,
    if_exists='replace'
)
print('done')

writing to: de_dev.data_science_editor_recommendation
done
